In [1]:
import json
import sys
import time
import urllib2
import subprocess
from IPython.core.debugger import set_trace

try:
    import requests
except ImportError:
    print "Please install the python-requests module."
    sys.exit(-1)

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)



In [2]:
URL = 'https://vm255-149.gsslab.pnq2.redhat.com/'
SAT_API = 'https://vm255-149.gsslab.pnq2.redhat.com/'

USERNAME = "admin"
PASSWORD = "redhat"

SSL_VERIFY = False   # Ignore SSL for now
def get_json(url):
    # Performs a GET using the passed URL location
    r = requests.get(url, auth=(USERNAME, PASSWORD), verify=SSL_VERIFY)
    return r.json()


In [3]:
def get_results(url):
    jsn = get_json(url)
    if jsn.get('error'):
        print "Error: " + jsn['error']['message']
    else:
        if jsn.get('results'):
            return jsn['results']
        elif 'results' not in jsn:
            return jsn
        else:
            print "No results found"
    return None

In [4]:
def post_json(location):
    """
    Performs a POST and passes the data to the URL location
    """
    POST_HEADERS = {'content-type': 'application/json'}
    result = requests.post(
        location,
        auth=(USERNAME, PASSWORD),
        verify=SSL_VERIFY,
        headers=POST_HEADERS)

    return result.json()

In [5]:
def wait_for_sync(seconds):
#   repo_sync_tasks = "foreman_tasks/api/tasks?search=utf8=%E2%9C%93&search=label+%3D+Actions%3A%3AKatello%3A%3ARepository%3A%3ASync+and+state+%3D+running"
    repo_sync_tasks = "foreman_tasks/api/tasks?utf8=.&search=Actions%3A%3AKatello%3A%3ARepository%3A%3ASync+and+state+%3D+running"
    """
    Wait for all repo sync tasks to terminate. Search string is:
    label = Actions::Katello::Repository::Sync and state = running
    """

    count = 0
    print "Waiting for repo sync tasks to finish..."

    # Make sure that repo sync tasks gets the chance to appear before looking for them
    time.sleep(2)

    while get_json(URL + repo_sync_tasks)["total"] != 0:
        time.sleep(seconds)
        count += 1

    print "Repo Sync completed waiting after " + str(seconds * count) + " seconds"


In [6]:
def display_task_results(url):
    results = get_results(url)
    if results:
        #print results['output']
        out = results['humanized']['output']
    return out

In [7]:
def main():
    date=(time.strftime("%d-%m-%Y"))

    url=SAT_API + '/katello/api/products/248/repositories?organization_id=1'
    results=get_results(url)
    length=0
    for result in results:
        length=length+1
    data = [{'RepoID' : 'repo_id', 'RepoName' : 'Name', 'SyncID' : 'sync_id', 'Output' : 'out'} for k in range(length)]
    i=0
    for result in results:
        sync_url=SAT_API + '/katello/api/repositories/%d/sync' % result['id']
        sync_out=post_json(sync_url)
        data[i]['RepoID'] = result['id']
        data[i]['SyncID'] = sync_out['id']
        data[i]['RepoName'] = result['name']
        time.sleep(60)
        i = i + 1
    wait_for_sync(10)
    set_trace()
    mail_file=open("mail.html","wb")
    mail_file.write("From: Automation Admin \n")
    mail_file.write("To: gkonda@redhat.com \n")
    mail_file.write("MIME-Version: 1.0 \n")
    mail_file.write("Subject: DR-(Satellite 6)RHEL-Server Product sync report ")
    mail_file.write(date)
    mail_file.write("\n")
    mail_file.write("Content-Type: text/html  \n")
    mail_file.write("<HTML><BODY><H3><CENTER>RHEL-Server Product sync report ")
    mail_file.write(date)
    mail_file.write(" </H3><HR><BR><TABLE border='1'>")
    mail_file.write("\n <TR bgcolor='#7FB3D5' align='center'><TD><B>Repository Name</B></TD><TD><B>Update</B></TD></TR> \n")

    for k in range(0,i):
        output=display_task_results(SAT_API + 'foreman_tasks/api/tasks/%s' % data[k]['SyncID'])
        mail_file.write("<TR><TD>")
        mail_file.write(data[k]['RepoName'])
        mail_file.write("</TD><TD>")
        mail_file.write(output)
        mail_file.write("</TD></TR> \n")

    mail_file.write("</TABLE></BODY></HTML>")
    print mail_file
    mail_file.close
    
    subprocess.Popen("/bin/cat mail.html | /usr/sbin/sendmail gkonda@redhat.com", shell=True)

In [ ]:
if __name__ == "__main__":
    main()

Waiting for repo sync tasks to finish...
Repo Sync completed waiting after 0 seconds
> <ipython-input-7-882762622938>(21)main()
     19     wait_for_sync(10)
     20     set_trace()
---> 21     mail_file=open("mail.html","wb")
     22     mail_file.write("From: Automation Admin \n")
     23     mail_file.write("To: gkonda@redhat.com \n")

ipdb> n
> <ipython-input-7-882762622938>(22)main()
     20     set_trace()
     21     mail_file=open("mail.html","wb")
---> 22     mail_file.write("From: Automation Admin \n")
     23     mail_file.write("To: gkonda@redhat.com \n")
     24     mail_file.write("MIME-Version: 1.0 \n")

ipdb> n
> <ipython-input-7-882762622938>(23)main()
     21     mail_file=open("mail.html","wb")
     22     mail_file.write("From: Automation Admin \n")
---> 23     mail_file.write("To: gkonda@redhat.com \n")
     24     mail_file.write("MIME-Version: 1.0 \n")
     25     mail_file.write("Subject: DR-(Satellite 6)RHEL-Server Product sync report ")

ipdb> n
> <ipython-inp

ipdb> n
> <ipython-input-7-882762622938>(31)main()
     29     mail_file.write("<HTML><BODY><H3><CENTER>RHEL-Server Product sync report ")
     30     mail_file.write(date)
---> 31     mail_file.write(" </H3><HR><BR><TABLE border='1'>")
     32     mail_file.write("\n <TR bgcolor='#7FB3D5' align='center'><TD><B>Repository Name</B></TD><TD><B>Update</B></TD></TR> \n")
     33 

ipdb> n
> <ipython-input-7-882762622938>(32)main()
     30     mail_file.write(date)
     31     mail_file.write(" </H3><HR><BR><TABLE border='1'>")
---> 32     mail_file.write("\n <TR bgcolor='#7FB3D5' align='center'><TD><B>Repository Name</B></TD><TD><B>Update</B></TD></TR> \n")
     33 
     34     for k in range(0,i):

ipdb> n
> <ipython-input-7-882762622938>(34)main()
     32     mail_file.write("\n <TR bgcolor='#7FB3D5' align='center'><TD><B>Repository Name</B></TD><TD><B>Update</B></TD></TR> \n")
     33 
---> 34     for k in range(0,i):
     35         output=display_task_results(SAT_API + 'foreman_tasks/

ipdb> n
> <ipython-input-7-882762622938>(42)main()
     40         mail_file.write("</TD></TR> \n")
     41 
---> 42     mail_file.write("</TABLE></BODY></HTML>")
     43     print mail_file
     44     mail_file.close

ipdb> n
> <ipython-input-7-882762622938>(43)main()
     41 
     42     mail_file.write("</TABLE></BODY></HTML>")
---> 43     print mail_file
     44     mail_file.close
     45 

ipdb> n
<open file 'mail.html', mode 'wb' at 0x2bbe0c0>
> <ipython-input-7-882762622938>(44)main()
     42     mail_file.write("</TABLE></BODY></HTML>")
     43     print mail_file
---> 44     mail_file.close
     45 
     46     subprocess.Popen("/bin/cat mail.html | /usr/sbin/sendmail gkonda@redhat.com", shell=True)

ipdb> n
> <ipython-input-7-882762622938>(46)main()
     42     mail_file.write("</TABLE></BODY></HTML>")
     43     print mail_file
     44     mail_file.close
     45 
---> 46     subprocess.Popen("/bin/cat mail.html | /usr/sbin/sendmail gkonda@redhat.com", shell=True)

ipdb> 